In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from codeStore import support_fun as spf
import importlib
PWD = os.getcwd()

## helix helicoid

In [11]:
use_node = 3
njob_node = 1
n_job_pbs = use_node * njob_node
random_order = True
n_job_pbs = 10000

# pythonmpi ../obj_helicoid_hlx.py -sm lg_rs_selfRotate -legendre_m 3 -legendre_k 2 -epsilon 3 
#  -rh1 1 -rh2 0.3 -eh 0 -ph 6.5 -nth 10 -ch 3 -hfct 1 -n_tail 1 -with_cover 2 -left_hand 0 
#  -main_resistanceMatrix_selfRotate 1 -ksp_max_it 1000 -f dbg 
#  -helicoid_ndsk_each 4 -helicoid_th0 0 -helicoid_r 100
def comm_obj_helicoid_dumb(sm, legendre_m, legendre_k, epsilon, 
                           rh1, rh2, nth, eh, ph, ch, n_tail, 
                           hfct, with_cover, with_T_geo, left_hand, 
                           helicoid_r, helicoid_th0, helicoid_ndsk_each, 
                           ksp_max_it, ksp_rtol, ksp_atol, f): 
    main_fun = 'main_resistanceMatrix_selfRotate' 
    ts = 'mpirun -np 24 python ../../obj_helicoid_hlx.py  -%s 1 ' % main_fun
    ts = ts + ' -sm %s -legendre_m %d -legendre_k %d -epsilon %f ' % (sm, legendre_m, legendre_k, epsilon)
    ts = ts + ' -rh1 %f -rh2 %f -nth %d -eh %f -ph %f -ch %f -n_tail %d ' % (rh1, rh2, nth, eh, ph, ch, n_tail)
    ts = ts + ' -hfct %f -with_cover %d -with_T_geo %d -left_hand %d ' % (hfct, with_cover, with_T_geo, left_hand)
    ts = ts + ' -helicoid_r %f -helicoid_th0 %f -helicoid_ndsk_each %d' % (helicoid_r, helicoid_th0, helicoid_ndsk_each)
    ts = ts + ' -ksp_max_it %d -ksp_rtol %e -ksp_atol %e -f %s' % (ksp_max_it, ksp_rtol, ksp_atol, f)
    return ts

write_pbs_head = spf.write_pbs_head
sm, legendre_m, legendre_k, epsilon = 'lg_rs_selfRotate', 3, 2, 3
rh1, rh2, nth, eh, n_tail = 1, 0.1, 11, -1, 1
hfct, with_cover, with_T_geo, left_hand = 1, 2, 0, 0
helicoid_th0, helicoid_ndsk_each, ksp_max_it, ksp_rtol, ksp_atol = 0, 4, 3000, 1e-20, 1e-200
ph_list = np.linspace(1, 10, 19)
# ch_list = np.array((0.1, 0.3, 0.5, 0.7, 0.9, 1, 1.5, 2, 2.5, 3, 3.5, 4))
ch_list = np.array((1, 1.5, 2, 2.5, 3, 3.5, 4))
helicoid_r_list = np.hstack((np.linspace(1, 9, 9), 10 ** np.linspace(1, 3, 30)))
job_dir = 'hlx_th0_a'
use_node = 20

###############################################################################################
# generate all comment file. 
comm_list = []
txt_list = []
helicoid_r_list, ph_list, ch_list = np.meshgrid(helicoid_r_list, ph_list, ch_list)
for helicoid_r, ph, ch in zip(helicoid_r_list.ravel(), ph_list.ravel(), ch_list.ravel()):
    f = 'c%.4f_ph%.4f_ch%.4f' % (helicoid_r, ph, ch)
    ts = comm_obj_helicoid_dumb(sm, legendre_m, legendre_k, epsilon, 
                                rh1, rh2, nth, eh, ph, ch, n_tail, 
                                hfct, with_cover, with_T_geo, left_hand, 
                                helicoid_r, helicoid_th0, helicoid_ndsk_each, 
                                ksp_max_it, ksp_rtol, ksp_atol, f)
    comm_list.append(ts)
    txt_list.append(f)
spf.write_main_run_comm_list(comm_list, txt_list, use_node, njob_node, job_dir, 
                             write_pbs_head, n_job_pbs, random_order=random_order)


make folder /home/zhangji/stokes_flow_master/HelicodsParticles/helicoid_hlx/hlx_th0_a
input 5187 cases.
generate 1 pbs files in total.
 --->>random order mode is ON. 
Command of first case is:
mpirun -np 24 python ../../obj_helicoid_hlx.py  -main_resistanceMatrix_selfRotate 1  -sm lg_rs_selfRotate -legendre_m 3 -legendre_k 2 -epsilon 3.000000  -rh1 1.000000 -rh2 0.100000 -nth 11 -eh -1.000000 -ph 6.000000 -ch 4.000000 -n_tail 1  -hfct 1.000000 -with_cover 2 -with_T_geo 0 -left_hand 0  -helicoid_r 280.721620 -helicoid_th0 0.000000 -helicoid_ndsk_each 4 -ksp_max_it 3000 -ksp_rtol 1.000000e-20 -ksp_atol 1.000000e-200 -f c280.7216_ph6.0000_ch4.0000


True

In [7]:
np.hstack((np.linspace(1, 9, 9), 10 ** np.linspace(1, 3, 30)))

array([   1.        ,    2.        ,    3.        ,    4.        ,
          5.        ,    6.        ,    7.        ,    8.        ,
          9.        ,   10.        ,   11.72102298,   13.73823796,
         16.10262028,   18.87391822,   22.12216291,   25.92943797,
         30.39195382,   35.6224789 ,   41.75318937,   48.93900918,
         57.3615251 ,   67.23357536,   78.80462816,   92.36708572,
        108.26367339,  126.89610032,  148.73521073,  174.33288222,
        204.33597179,  239.502662  ,  280.72162039,  329.03445623,
        385.66204212,  452.03536564,  529.83169063,  621.01694189,
        727.8953844 ,  853.16785242, 1000.        ])